In [14]:
import subprocess
import os
import tempfile
import random
import uuid
import numpy as np
import hashlib
from tqdm import tqdm

In [58]:
def ci(tmp):
    # print(tmp)
    return [int(x) for x in tmp.split(" ")]

In [203]:
def brtf(inp):
    inp = inp.split("\n")
    T,x0 = ci(inp[0])
    TRs = []
    inputtic = 1
    for i in range(T):
        r = ci(inp[inputtic])[0]
        inputtic += 1
        c1,c2,m = ci(inp[inputtic])
        inputtic += 1
        a,b,c,d = ci(inp[inputtic])
        inputtic += 1
        TRs.append((r,np.array([[a,b],[c,d]],dtype = np.uint64),c1,c2,m))
    
    def check(x1):
        X = np.array([[x0],[x1]],dtype = np.uint64)
        # print(X)
        for ri,T,c1,c2,m in TRs:
            # print(ri,T,c1,c2,m)
            # print(X,T)
            # print(T @ X)
            for _ in range(ri):
                X = T @ X
                # print(X)
            # print(int(X[0][0]) % m ,int(X[1][0]) % m)
            if(int(X[0][0])  < 0 or int(X[1][0]) < 0 or 
                int(X[0][0]) > 1e18 or int(X[1][0]) > 1e18 ):
                raise Exception("Out of Range")
            if int(X[0][0]) % m == int(c1) and int(X[1][0]) % m == int(c2) :
                pass
            else :
                return 0 
        return 1

    # print(check(10))
    
    
    for x in range(0,int(1e5)):
        if(check(x)) : return x
    
    return -1

In [27]:
X

NameError: name 'X' is not defined

In [204]:
brtf("""5 4
5
8 8 9
10 1 10 10
3
0 0 4
9 6 9 5
4
0 0 5
5 6 6 1
1
6 1 9
9 6 3 10
5
3 6 7
7 3 6 9""")

Exception: Out of Range

In [52]:
def run(
    cpp_filename: str,
    input_str: str = "",
    timeout: int = 3,
    force_recompile: bool = False
) -> str:
    """
    编译并运行C++程序，支持增量编译
    
    参数:
        input_str: 输入到程序的字符串
        cpp_filename: C++源代码文件路径
        timeout: 程序运行超时时间（秒）
        force_recompile: 是否强制重新编译（默认False）
    
    返回:
        str: 程序的标准输出内容
        
    异常:
        RuntimeError: 如果编译失败或程序运行超时
    """
    # 生成唯一但稳定的可执行文件路径
    abs_cpp_path = os.path.abspath(cpp_filename)
    path_hash = hashlib.md5(abs_cpp_path.encode()).hexdigest()
    exe_path = os.path.join(tempfile.gettempdir(), f"cpp_{path_hash}")

    try:
        # 编译条件判断
        if force_recompile or not os.path.exists(exe_path):
            # 编译C++程序
            compile_proc = subprocess.run(
                ["g++", cpp_filename, "-o", exe_path],
                capture_output=True,
                text=True
            )
            
            if compile_proc.returncode != 0:
                if os.path.exists(exe_path):
                    os.remove(exe_path)  # 清理无效的可执行文件
                raise RuntimeError(f"编译失败：\n{compile_proc.stderr}")

        # 运行程序
        run_proc = subprocess.run(
            [exe_path],
            input=input_str,
            capture_output=True,
            text=True,
            timeout=timeout
        )
        if len(run_proc.stderr) != 0:
            print("ERR:" + run_proc.stderr)
        return run_proc.stdout

    except subprocess.TimeoutExpired:
        # raise RuntimeError(f"程序{cpp_filename}运行超时（{timeout}秒）")
        if(cpp_filename == "gnc.cpp"):
            print(f"测试点生成失败")
        else :
            print("TLE")
        # print(f"{input_str}")
        return ""
        
    # except Exception as e:
        # if os.path.exists(exe_path):
        #     os.remove(exe_path)  # 发生异常时清理可执行文件
        # raise

In [53]:
seed = 113
ans = 11
n = 1
CPS = 10

In [54]:
def tgc(cin,cout,T):
    IN = open(f"{T}.in","w")
    OUT = open(f"{T}.out","w")
    IN.write(cin)
    OUT.write(cout)
    IN.close()
    OUT.close()

In [211]:
# compile
run(cpp_filename="gnc.cpp",force_recompile=1)
run(cpp_filename="std.cpp",force_recompile=1)
# run(cpp_filename="brt.cpp",force_recompile=1)

''

In [207]:
for i in tqdm(range(10000)):
    key = np.random.randint(1,100000000)
    ans = np.random.randint(1,1000)
    n = 3
    CPS = 10

    tc = run(
        cpp_filename = "gnc.cpp",
        input_str = f"{key} {ans} {n} {CPS}",
        timeout = 1
    )
    stdres = run(
        cpp_filename = "std.cpp",
        input_str = tc,
        timeout = 1
    )
    brtres = brtf(tc)
    if(int(stdres) != int(brtres)) :
        print(f"WA at:{key}")
        print(tc)
        print(f"-------\nstd:{stdres},brt:{brtres},exp{ans}")
        break

100%|██████████| 10000/10000 [00:31<00:00, 316.39it/s]


In [ ]:
for i in tqdm(range(10000)):
    key = np.random.randint(1,100000000)
    ans = -1
    n = 3
    CPS = 10

    tc = run(
        cpp_filename = "gnc.cpp",
        input_str = f"{key} {ans} {n} {CPS}",
        timeout = 1
    )
    stdres = run(
        cpp_filename = "std.cpp",
        input_str = tc,
        timeout = 1
    )
    brtres = brtf(tc)
    if(int(stdres) != int(brtres)) :
        print(f"WA at:{key}")
        print(tc)
        print(f"-------\nstd:{stdres},brt:{brtres},exp{ans}")
        break

In [144]:
print(run(
    cpp_filename = "std.cpp",
    input_str = """
1 34
1
2 6 30
8 10 3 26
""",
    force_recompile = 1
))

9



In [141]:
(12*13)%15

6

In [85]:
X = np.array([[10,1],[5,1]])
X @ X
X[0][0] %= 9
X[0][1] %= 9
X[1][0] %= 9
X[1][1] %= 9
X

array([[1, 1],
       [5, 1]])

In [208]:
# for 0
T = 0

CPS = int(1e9)

# while True:

for i in tqdm(range(5)):
    T += 1
    key = np.random.randint(1,1145140000)
    ans = 0
    n = 500
    tc = run(
        cpp_filename = "gnc.cpp",
        input_str = f"{key} {ans} {n} {CPS}"
    )
    if(tc == "") :
        T -= 1
        continue
    res = run(
        cpp_filename = "std.cpp",
        input_str = tc
    )
    # if int(res) != ans:
    #     T -= 1
    #     print(f"WA at:{tc}")
    #     break
    # else :
    tgc(tc,res,T)
        # pass
        # print(f"pass testcases {T}")

100%|██████████| 5/5 [00:17<00:00,  3.49s/it]


In [210]:
# for rand value
for i in tqdm(range(5)):
    T += 1
    key = np.random.randint(1,1145140000)
    ans = np.random.randint(1,100000000)
    n = 500
    # n = 1
    tc = run(
        cpp_filename = "gnc.cpp",
        input_str = f"{key} {ans} {n} {CPS}"
    )
    if(tc == "") :
        T -= 1
        continue
    res = run(
        cpp_filename = "std.cpp",
        input_str = tc
    )
    # if int(res) != ans:
    #     T -= 1
    #     print(f"WA at:{tc}")
    #     break
    # else :
    tgc(tc,res,T)
    pass
        # print(f"pass testcases {T}")

100%|██████████| 5/5 [00:17<00:00,  3.47s/it]


In [212]:
# for rand value
for i in tqdm(range(5)):
    T += 1
    key = np.random.randint(1,1145140000)
    ans = -1
    n = 500
    # n = 1
    tc = run(
        cpp_filename = "gnc.cpp",
        input_str = f"{key} {ans} {n} {CPS}"
    )
    if(tc == "") :
        T -= 1
        continue
    res = run(
        cpp_filename = "std.cpp",
        input_str = tc
    )
    # if int(res) != ans:
    #     T -= 1
    #     print(f"WA at:{tc}")
    #     break
    # else :
    tgc(tc,res,T)
    pass
        # print(f"pass testcases {T}")

100%|██████████| 5/5 [00:08<00:00,  1.80s/it]


In [29]:
ans

53205525

In [ ]:
if "YES" not in "1 24131YESYES":
    print(111)

In [26]:
# while True:
print(
    run(
        cpp_filename="std.cpp",
        input_str=run(cpp_filename="gnc.cpp",input_str=f"{key} {ans} {n}\n",timeout=3,force_recompile=1),
        timeout = 1,
        force_recompile=1
    )
)

TLE



In [10]:
ans

0

In [47]:
print(run(cpp_filename="gnc.cpp",input_str=f"{seed} {ans} {n} {CPS}\n",timeout=3,force_recompile=1))

1 2
2
0 0 1
9 10 5 3



In [32]:
run(cpp_filename="gnc_data.cpp",input_str=f"{CPS} NO {np.random.randint(1,100000)}\n",timeout=3,force_recompile=0)

测试点1生成失败
20 NO 38462



'Yes\nNO\n'

In [44]:
print(run(cpp_filename="std.cpp",input_str="""1 2
2
4 0 4
1 1 4 5
""",timeout=1,
force_recompile = 1
))

0 + 2 *  x  === 2 ( mod 4)
0 + 1 *  x  === 0 ( mod 4)
---------
0 + 1 *  x  === 1 ( mod 4)
0 + 1 *  x  === 0 ( mod 4)
---------
-1



In [76]:
import random
import pandas as pd

# Miller-Rabin primality test
def is_prime(n):
    if n < 2:
        return False
    small_primes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
    for p in small_primes:
        if n == p:
            return True
        if n % p == 0:
            return False
    # write n-1 as d*2^s
    d, s = n - 1, 0
    while d % 2 == 0:
        d //= 2
        s += 1
    # test bases
    test_bases = [2, 325, 9375, 28178, 450775, 9780504, 1795265022]
    for a in test_bases:
        if a % n == 0:
            continue
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(s - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

# Generate 50 representative positions in [0, 1e9]
positions = [int(i * (10**9) / 49) for i in range(50)]

# Find the next prime at or after each position
primes = []
for pos in positions:
    x = max(2, pos)
    while x <= 10**9:
        if is_prime(x):
            primes.append(x)
            break
        x += 1
    else:
        primes.append(None)

# Create a DataFrame and display
df = pd.DataFrame({
    'Index': list(range(1, 51)),
    'Position': positions,
    'Prime': primes
})
# import ace_tools as tools; tools.display_dataframe_to_user(name="50 Representative Primes in [0, 1e9]", dataframe=df)
print(df)


    Index    Position        Prime
0       1           0          2.0
1       2    20408163   20408189.0
2       3    40816326   40816387.0
3       4    61224489   61224523.0
4       5    81632653   81632653.0
5       6   102040816  102040843.0
6       7   122448979  122449007.0
7       8   142857142  142857151.0
8       9   163265306  163265327.0
9      10   183673469  183673493.0
10     11   204081632  204081637.0
11     12   224489795  224489803.0
12     13   244897959  244897979.0
13     14   265306122  265306133.0
14     15   285714285  285714287.0
15     16   306122448  306122449.0
16     17   326530612  326530619.0
17     18   346938775  346938791.0
18     19   367346938  367346977.0
19     20   387755102  387755117.0
20     21   408163265  408163271.0
21     22   428571428  428571431.0
22     23   448979591  448979593.0
23     24   469387755  469387817.0
24     25   489795918  489795919.0
25     26   510204081  510204109.0
26     27   530612244  530612263.0
27     28   55102040

In [ ]:
# 对于方程 ax === b (%m)
